In [ ]:
import numpy
import cv2
import PIL.Image
import skgeom as sg

def _normalized(v):
    return v / numpy.linalg.norm(v)

def generate_random_polyline_mask(n=15, size=512, thickness=2):
    pixels = numpy.zeros((size, size), dtype=numpy.uint8)

    while True:
        poly = sg.random_polygon(
            n, size=size // 4, shape="disc")

        line = [(
            int(float(p.x() + size // 2)),
            int(float(p.y() + size // 2)))
            for p in list(poly.vertices)]

        pts = numpy.array(line, dtype=numpy.int32)
        dots = [
            numpy.dot(_normalized(v - u), _normalized(z - v))
            for u, v, z in zip(pts, pts[1:], pts[2:])]

        if min(dots) > 0:
            break

    for u, v in zip(line, line[1:]):
        cv2.line(pixels, u, v, color=255, thickness=thickness)
        
    return pixels

pixels = generate_random_polyline_mask()
PIL.Image.fromarray(pixels, "L")

In [ ]:
import mask_to_polyline
polylines = mask_to_polyline.find_polylines(pixels)

def draw_polyline(pixels, polylines):
    import shapely.geometry
    pixels2 = (255 - (pixels.copy() // 2))
    pixels2 = numpy.array(PIL.Image.fromarray(pixels2, "L").convert("RGB"))
    for polyline in polylines:

        coords = polyline.coords
        s = shapely.geometry.LineString(coords).simplify(2)
        coords = list(s.coords)

        for u, v in zip(coords, coords[1:]):
            u = tuple(map(int, u))
            v = tuple(map(int, v))
            cv2.line(pixels2, u, v, color=(230, 0, 100), thickness=1)

        for p in coords:
            p = tuple(map(int, p))
            cv2.circle(pixels2, p, 3, color=(100, 150, 200), thickness=-1)

    return PIL.Image.fromarray(pixels2)
    
draw_polyline(pixels, polylines)